In [ ]:
import pandas as pd
import random
import numpy as np
import sys
import os
import shutil
import glob

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

from PIL import Image
from skimage import morphology
import matplotlib.pyplot as plt
from skan import draw
from skan.csr import skeleton_to_csgraph
from skan import Skeleton, summarize
from skan.pre import threshold

In [ ]:
props_ranges = {
    # Image configuration
    "GRID_SIZE": [500],
    "ANGLE_LOWER_BOUND": list(range(1, 30)),
    "ANGLE_UPPER_BOUND": list(range(30, 90)),
    "TORTUOUS_MOVEMENT_LENGTH_LIMITER": list(np.arange(0.1, 1.1, 0.1)),
    "TORTUOUS_PROBABILITY": list(np.arange(0.1, 1.1, 0.1)),
    "MOVEMENT_LENGTH_LIMITER": list(np.arange(0.1, 1.1, 0.1)),
    "NUM_WALKERS": list(range(1, 21)),
    "MAX_MOVES": list(range(1, 26)),
    "WALKER_MATURITY_STEPS": list(range(10)),
    "WALKER_INITIAL_REPRODUCTION_PROBABILITY": list(np.arange(0.1, 1.1, 0.1)),
    "WALKER_CHILD_REPRODUCTION_PROBABILITY_MULTIPLIER": list(np.arange(0.1, 5.1, 0.1)),
    "WALKER_INITIAL_DEATH_PROBABILITY": list(np.arange(0, 1, 0.001)),
    "WALKER_CHILD_DEATH_PROBABILITY_MULTIPLIER": list(np.arange(1, 600, 1)),
    "WALKER_INITIAL_PATH_WIDTH": [0.005],
    "WALKER_PATH_WIDTH_DECAY": [0.01],
    "VECTOR_FIELD_WEIGHT": list(np.arange(0.1, 1.1, 0.05)),
    "MIDDLE_LINE_WEIGHT": list(np.arange(0.1, 1.1, 0.05)),
    "SINK_STRENGTH": list(np.arange(1, 10, 0.5)),
}

In [ ]:
def create_individual(cls):    
    individual = {}
    for (attr, values) in props_ranges.items():
        individual[attr] = np.random.choice(values)
        
    return cls(individual)

In [ ]:
def open_and_convert_to_grayscale_np(path):
    img = Image.open(path)
    img = img.convert('L')
    img = np.asarray(img)
    smooth_radius = 0.1
    threshold_radius = 10
    img = threshold(img, sigma=smooth_radius, radius=threshold_radius)
    return img

In [ ]:
def get_info_image(image_path):
    img = open_and_convert_to_grayscale_np(image_path)
    skeleton = morphology.skeletonize(img)
    skeleton = Skeleton(skeleton)
    branch_data = summarize(skeleton)
    data = {
        "mean_branch_length": branch_data["branch-distance"].mean(),
        "num_branches": branch_data["branch-distance"].count(),
        **{f"branch_types_{type}": freq for (type, freq) in branch_data["branch-type"].value_counts().to_dict().items()},
    }
    return skeleton, data

    

In [ ]:
def get_metrics(df):
    return [
        df['mean_branch_length'].mean(),
        df['num_branches'].mean(),
#         (df['branch_types_0']/df['num_branches']).mean(),
#         (df['branch_types_1']/df['num_branches']).mean(),
#         (df['branch_types_2']/df['num_branches']).mean(),
#         (df['branch_types_3']/df['num_branches']).mean()
    ]

In [ ]:
def fitness_error(individual):
    with open("config.py", "w") as f:
        for (attr, value) in individual.items():
            print(attr, "=", value, file=f)
            
    shutil.rmtree("images", ignore_errors=True)
    os.system("python main.py")
    
    GENERATED_IMAGES_PATH = glob.glob("../SyntheticDataset/images/*/*.png")    
    generated_d = {}
    for img_path in GENERATED_IMAGES_PATH:
        skeleton, data = get_info_image(img_path)
        data = {
            "type": "generated", 
            **data
        }
        generated_d[img_path] = data
    generated_df = pd.DataFrame.from_dict(generated_d, orient='index')
    generated_df['filename'] = generated_df.index
    generated_df.reset_index(drop=True, inplace=True)
    generated_df.fillna(0, inplace=True)
        
    curr_metrics = get_metrics(generated_df)
    
    with open("../SkeletonAnalysis/original_metrics.txt", "r") as f:
        original_metrics = list([float(line.strip()) for line in f])
        
    error = 0
    for (m1, m2) in zip(curr_metrics, original_metrics):
        error += ((abs(m2 - m1)/abs(m1)) ** 2)
    return error,
        

In [ ]:
def crossover(ind1, ind2):
    child1 = {}
    child2 = {}
    for (attr, value) in ind1.items():
        if random.random() > 0.5:
            child1[attr] = value
            child2[attr] = ind2[attr]
        else:
            child1[attr] = ind2[attr]
            child2[attr] = value
    return creator.Individual(child1), creator.Individual(child2)

In [ ]:
def mutate(individual):
    # attr = random.choice(list(individual.keys()))
    # individual[attr] = np.random.choice(props_ranges[attr])
    for attr in individual.keys():
        if random.random() < 0.2:
            individual[attr] = np.random.choice(props_ranges[attr])
    return individual,

In [ ]:
creator.create("Fitness", base.Fitness, weights=(-1.0,))
creator.create("Individual", dict, fitness=creator.Fitness)

In [ ]:
toolbox = base.Toolbox()

In [ ]:
toolbox.register("individual", create_individual, creator.Individual)

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
toolbox.register("evaluate", fitness_error)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selNSGA2)

In [ ]:
%%time
random.seed(64)
NGEN = 20
NUM_INDS = 50
CXPB = 0.7
MUTPB = 0.2

pop = toolbox.population(n=NUM_INDS)
hof = tools.HallOfFame(maxsize=3)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

algorithms.eaSimple(pop, toolbox, CXPB, MUTPB, NGEN, stats, halloffame=hof, verbose=True)

In [ ]:
print(pop)
print(stats)
print(hof)

In [ ]:
best = hof[0]

In [ ]:
with open("config_best.py", "w") as f:
    for (attr, value) in best.items():
        print(attr, "=", value, file=f)

In [ ]:
best

In [ ]:
fitness_error(best)